<a href="https://colab.research.google.com/github/veereshthotigar/CSEE5590-490-AI-CyberSecurity-/blob/master/ICP66/Membership_Attack_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# required imports
import sys 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline  

import pickle
import torch
import torchvision 
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
# mount the google drive to download the datasets
from google.colab import drive
drive.mount('/content/drive')
project_path = '/content/drive/My Drive/cybersecurity'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:

#load the dataset
with open(project_path+'/data/shadow.data', 'rb') as filehandle:
    # read the data as binary data stream
    predictionsList = pickle.load(filehandle)

In [41]:
    
total_size = len(predictionsList)
split1 = total_size // 3

print(total_size, split1)

indices = list(range(total_size))

# two groups to train the shadow (in and out)
train_idx = indices[:split1]
test_idx = indices[split1:]

batch_size = 32 # pick your own

# divide and load shadow train in and out
train_sampler = SubsetRandomSampler(train_idx) # Pytorch function
train_loader = torch.utils.data.DataLoader(predictionsList, batch_size=batch_size, sampler=train_sampler)

test_sampler = SubsetRandomSampler(test_idx) # Pytorch function
test_loader = torch.utils.data.DataLoader(predictionsList, batch_size=batch_size, sampler=test_sampler)   


25000 8333


In [42]:
# create the Attack Model: A NN binary classifier {0, 1}
# the input to this model is the propability distribution vector of size 10
# and the output is either 0 (input was not included in training) or 1
from torch.autograd import Variable
attack_model = nn.Sequential(nn.Linear(10, 20),
                      nn.ReLU(),
                      nn.Linear(20, 26),
                      nn.ReLU(),
                      nn.Linear(26, 16),
                      nn.ReLU(),
                      nn.Linear(16, 8),
                      nn.ReLU(),
                      nn.Linear(8, 2),
                      nn.LogSoftmax(dim=1))
attack_model = attack_model.cuda()
attack_criterion = nn.CrossEntropyLoss()
attack_optimizer = optim.Adam(attack_model.parameters(), lr=0.001)

epochs = 20
for e in range(epochs):
    running_loss = 0
    for images, labels in train_loader:
        # sending tensors to GPU
        #images = torch.from_numpy(images).cuda().float()
        #labels = torch.from_numpy(labels).cuda().long()
        images = images.cuda().float()
        labels = labels.cuda().long()
        attack_optimizer.zero_grad()
        output = attack_model(images)
        #output = torch.argmax(output,dim=1)
        #print(output,labels)
        loss = attack_criterion(output, labels)
        loss.backward()
        attack_optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(predictionsList)}")

Training loss: 0.0011884193064819555
Training loss: 9.109104216986452e-07
Training loss: 7.488049640414829e-08
Training loss: 2.3670405043958452e-08
Training loss: 9.448890685916922e-09
Training loss: 2.31645407779979e-09
Training loss: 8.485408234548686e-10
Training loss: 2.801418078846041e-11
Training loss: 0.0
Training loss: 0.0
Training loss: 0.0
Training loss: 0.0
Training loss: 0.0
Training loss: 0.0
Training loss: 0.0
Training loss: 0.0
Training loss: 0.0
Training loss: 0.0
Training loss: 0.0
Training loss: 0.0


In [43]:
# calculate the accuracy of the attack Model
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        # sending tensors to GPU
        images = images.cuda().float()
        labels = labels.cuda().long()
        outputs = attack_model(images)
        _, predicted = torch.max(outputs.data, 1)
        #print(predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 12500 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 12500 test images: 25 %


In [0]:
#load the dataset
with open(project_path+'/data/target.data', 'rb') as filehandle:
    # read the data as binary data stream
    validation = pickle.load(filehandle)

In [0]:
batch_size = 32 # pick your own

validation_loader = torch.utils.data.DataLoader(validation, batch_size=batch_size)

In [49]:
#input the output of the target model to your attack network 
# you already know the target_in and target_out samples, so use that info to evaluate the attack model
correct = 0
total = 0

with torch.no_grad():
    for images, labels in validation_loader:
        # sending tensors to GPU
        images = images.cuda().float()
        labels = labels.cuda().long()
        outputs = attack_model(images)
        _, predicted = torch.max(outputs.data, 1)
        #print(predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 12500 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 12500 test images: 50 %


Great! At this point, you must have created a succesfful attack model that can detect whether a datapoint was used in training a target mode or not. 
* A successful attack model is one with a precision/recall higher than 85% -- you are using same architecture and are aware of the data classes

 
 Can you suggest any defense mechanism? If yes, Apply them to your solution and re-evaluate your attack model. How did your defense mecanism affect the accuracy of the target model? How did it affect the recall and precision of the Attack model?